SetUp and Data Preprocessing

In [ ]:
import os
import random
import tarfile
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, ConcatDataset, Subset
import torchvision.transforms as transforms
from PIL import Image
from timm import create_model

# --- Utility to Extract TAR Files Safely ---
def extract_tar(file_path, extract_to):
    with tarfile.open(file_path, 'r') as tar:
        def is_within_directory(directory, target):
            abs_directory = os.path.abspath(directory)
            abs_target = os.path.abspath(target)
            return abs_target.startswith(abs_directory)

        def safe_extract(tar, path=".", members=None):
            for member in tar.getmembers():
                member_path = os.path.join(path, member.name)
                if not is_within_directory(path, member_path):
                    raise Exception("Attempted Path Traversal in Tar File")
            tar.extractall(path, members=members)

        safe_extract(tar, path=extract_to)

# --- Validate Directories ---
def validate_directory(path, name):
    if not os.path.isdir(path):
        raise FileNotFoundError(f"{name} directory not found: {path}")

# --- Custom Dataset ---
class CustomDataset(Dataset):
    def __init__(self, data_dir, label, transform=None):
        self.data_dir = data_dir
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, self.label

# --- Data Preparation ---
train_real_dir = r'D:\SPS\train_real\real'
train_fake_dir = r'D:\SPS\train_fake\fake'
valid_real_dir = r'D:\SPS\valid_real\real'
valid_fake_dir = r'D:\SPS\valid_fake\fake'

# Validate directories
validate_directory(train_real_dir, "Train Real")
validate_directory(train_fake_dir, "Train Fake")
validate_directory(valid_real_dir, "Valid Real")
validate_directory(valid_fake_dir, "Valid Fake")

# Define transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load datasets
train_real_dataset = CustomDataset(train_real_dir, label=0, transform=train_transforms)
train_fake_dataset = CustomDataset(train_fake_dir, label=1, transform=train_transforms)
val_real_dataset = CustomDataset(valid_real_dir, label=0, transform=val_transforms)
val_fake_dataset = CustomDataset(valid_fake_dir, label=1, transform=val_transforms)

# Combine datasets
train_dataset = ConcatDataset([train_real_dataset, train_fake_dataset])
val_dataset = ConcatDataset([val_real_dataset, val_fake_dataset])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# --- Model Definition ---
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=1)
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 1),
    nn.Sigmoid()
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# --- Training and Validation Functions ---
def train_model(model, train_loader, val_loader, epochs=10, optimizer=optimizer, criterion=criterion):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predictions = (outputs > 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = (correct_predictions / total_predictions) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")
        validate_model(model, val_loader)

def validate_model(model, val_loader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device).float()
            outputs = model(inputs)
            predictions = (outputs > 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    val_accuracy = (correct_predictions / total_predictions) * 100
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# --- Predict Function ---
def predict_image(model, img_path):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    img = val_transforms(img).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(img)
        prediction = (output > 0.5).float().item()
    return "Fake" if prediction == 1 else "Real"

# --- Train and Test ---
train_model(model, train_loader, val_loader, epochs=10)

# Test with a sample image
img_path = r'D:\SPS\valid_real\real\valid_real_0029961.png'
result = predict_image(model, img_path)
print(f"The image is classified as: {result}")


In [ ]:
!pip install Pillow


Data Augmentation and Transformation

In [ ]:
import os
import torch
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from PIL import Image

# Define dataset paths
train_real_dir = r'D:\SPS\train_real\real'
train_fake_dir = r'D:\SPS\train_fake\fake'
valid_real_dir = r'D:\SPS\valid_real\real'
valid_fake_dir = r'D:\SPS\valid_fake\fake'

# Validate directories
def validate_directory(path, name):
    if not os.path.isdir(path):
        raise FileNotFoundError(f"{name} directory not found: {path}")

validate_directory(train_real_dir, "Train Real")
validate_directory(train_fake_dir, "Train Fake")
validate_directory(valid_real_dir, "Valid Real")
validate_directory(valid_fake_dir, "Valid Fake")

# Custom Dataset Class
class CustomDataset(Dataset):
    def __init__(self, data_dir, label, transform=None):
        self.data_dir = data_dir
        self.label = label
        self.transform = transform
        self.image_paths = [os.path.join(data_dir, img) for img in os.listdir(data_dir)]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = Image.open(img_path).convert("RGB")  # Ensure images are RGB
        if self.transform:
            img = self.transform(img)
        return img, self.label

# Define Data Transformations
train_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Load training data
train_real_dataset = CustomDataset(train_real_dir, label=0, transform=train_transforms)
train_fake_dataset = CustomDataset(train_fake_dir, label=1, transform=train_transforms)

# Load validation data
val_real_dataset = CustomDataset(valid_real_dir, label=0, transform=val_transforms)
val_fake_dataset = CustomDataset(valid_fake_dir, label=1, transform=val_transforms)

# Combine datasets
train_dataset = torch.utils.data.ConcatDataset([train_real_dataset, train_fake_dataset])
val_dataset = torch.utils.data.ConcatDataset([val_real_dataset, val_fake_dataset])

# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Verify data loading
for images, labels in train_loader:
    print(f"Batch images shape: {images.shape}")
    print(f"Batch labels: {labels}")
    break


In [ ]:
python -m pip3 install --upgrade pip




In [ ]:
!pip install timm


Load Pre-Trained ViT Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model

# Model Creation
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=1)

# Modify the model head for binary classification (sigmoid output)
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 1),  # One output unit
    nn.Sigmoid()  # Sigmoid activation for binary classification
)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Loss function (binary cross-entropy)
criterion = nn.BCELoss()

# Optimizer (Adam optimizer)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Verify model, loss function, and optimizer
print(model)
print(criterion)
print(optimizer)


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model
from torch.utils.data import DataLoader, Subset
import random

# Define the model with binary classification output
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=1)
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 1),
    nn.Sigmoid()
)

# Define the device, criterion, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Randomly select a subset from the dataset
def get_random_subset(dataset, subset_size=300):
    total_samples = len(dataset)
    subset_indices = random.sample(range(total_samples), subset_size)
    return Subset(dataset, subset_indices)

# Training function
def train_model(model, train_loader, val_loader, epochs=10, optimizer=optimizer, criterion=criterion):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Training phase
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predictions = (outputs > 0.5).float()
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

        # Epoch statistics
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = (correct_predictions / total_predictions) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation phase
        validate_model(model, val_loader)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
           # Inside the training loop
            predictions = (outputs.view(-1) > 0.5).float()  # Reshape outputs to 1D and threshold
            correct_predictions += (predictions == labels.float()).sum().item()  # Ensure labels are float
            total_predictions += labels.size(0)  # Total number of samples


    val_accuracy = (correct_predictions / total_predictions) * 100
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# Prepare DataLoader for a subset
subset_size = 300
train_subset = get_random_subset(train_dataset, subset_size=subset_size)  # Ensure train_dataset is defined
train_subset_loader = DataLoader(train_subset, batch_size=32, shuffle=True)

# Train the model using the subset DataLoader
train_model(model, train_subset_loader, val_loader, epochs=10)


Training the Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from timm import create_model

# Define the model with binary classification output
model = create_model('vit_base_patch16_224', pretrained=True, num_classes=1)
model.head = nn.Sequential(
    nn.Linear(model.head.in_features, 1),
    nn.Sigmoid()
)

# Define the device, criterion, and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train_model(model, train_loader, val_loader, epochs=10, optimizer=optimizer, criterion=criterion):
    model.train()

    for epoch in range(epochs):
        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        # Training phase
        for batch_idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())  # Ensure output matches the label shape

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            predictions = (outputs > 0.5).float()  # Convert outputs to binary predictions
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

            # Print the processed images count every 100 batches
            if (batch_idx + 1) % 100 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Processed {total_predictions}/{len(train_loader.dataset)} images")

        # Calculate epoch loss and accuracy
        epoch_loss = running_loss / len(train_loader)
        epoch_accuracy = (correct_predictions / total_predictions) * 100
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%")

        # Validation phase
        validate_model(model, val_loader)

# Validation function
def validate_model(model, val_loader):
    model.eval()
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predictions = (outputs > 0.5).float()  # Convert outputs to binary predictions
            correct_predictions += (predictions == labels).sum().item()
            total_predictions += labels.size(0)

    val_accuracy = (correct_predictions / total_predictions) * 100
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

# Training the model
train_model(model, train_loader, val_loader, epochs=10)


Evaluate the model

In [ ]:
def predict_image(model, img_path):
    model.eval()
    img = Image.open(img_path).convert("RGB")
    img = val_transforms(img).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(img)
        prediction = (output > 0.5).float().item()

    return "Fake" if prediction == 1 else "Real"


img_path = r'D:\SPS\valid_real\real\valid_real_0029961.png'
result = predict_image(model, img_path)
print(f"The image is classified as: {result}")
